In [62]:
import numpy as np
import matplotlib.pyplot as plt

In [63]:
#PARAMETROS DO MODELO
D = 0.0039
P = 0.0453
C0 = 8e3
ALPHA = 0.0305
AB = 10

#DOMINIO DO MODELO
L = 20
TF = 80

#DOMÍNIO DISCRETIZADO
N_S = 3
N_T = TF

#DOMINIO
SPATIAL_DOM = (-1, 1)
TIME_DOM = (0, TF * P)

In [69]:
#DISCRETIZAÇÃO DO DOMÍNIO
X = np.linspace(-1, 1, N_S)
Y = np.linspace(-1, 1, N_S)

#FUNÇÃO PARA REALIZAR A COMBINAÇÃO ENTRE DOIS ARRAYS
def Combination(x, y):

    comb = []
    for i in x:
        for j in y: 
            comb.append([i, j])

    return np.array(comb)

#FUNÇÃO PARA RETORNAR VALORES DA CONDIÇÃO INICIAL PARA UMA ENTRADA BIDIMENSIONAL
def InitialCondition(i): 
    
    return L**3 * np.exp(-100 * (i[:,0]**2 + i[:,1]**2))

#OBTENÇÃO DO VETOR X_K DA CONDIÇÃO INICIAL
INPUT = Combination(X, Y)
X_K = InitialCondition(INPUT)

#IMPLEMENTAÇÃO DA MATRIZ DOS COEFICIENTES

dx, dy = X[1], Y[1]

A = np.zeros((N_S**2, N_S**2))
A[0,0] =  - 3 / (4 * dx)
A[0, 1] = 1 / dx
A[0, 2] = -1 / (4 * dx)

A[-1, -1] = 3 / (4 * dx)
A[-1, -2] = -1 / dx
A[-1, -3] = 1 / (4 * dx)

A[N_S-1, N_S-1-2] =  3 / (4 * dx)
A[N_S-1, N_S-1-1] = -1 / dx
A[N_S-1, N_S-1] = 1 / (4 * dx)

A[-N_S, -N_S] =  -3 / (4 * dx)
A[-N_S, -N_S+1] = 1 / dx
A[-N_S, -N_S+2] = -1 / (4 * dx)


for i in range(1, N_S - 1):
    A[i, i] = - 3 / (4 * dx)
    A[i, i+1] = 1 / dx
    A[i, i+2] = -1 / (4 * dx)

    A[-i-1, -i] =  3 / (4 * dx)
    A[-i-1, -i-1] = -1 / dx
    A[-i-1, -i-2] = 1 / (4 * dx)

print(A)

[[-inf  inf -inf   0.   0.   0.   0.   0.   0.]
 [  0. -inf  inf -inf   0.   0.   0.   0.   0.]
 [ inf -inf  inf   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0. -inf  inf -inf]
 [  0.   0.   0.   0.   0.   0.  inf -inf  inf]
 [  0.   0.   0.   0.   0.   0.  inf -inf  inf]]


/tmp/ipykernel_622/1167035978.py:29: RuntimeWarning: divide by zero encountered in scalar divide
  A[0,0] =  - 3 / (4 * dx)
/tmp/ipykernel_622/1167035978.py:30: RuntimeWarning: divide by zero encountered in scalar divide
  A[0, 1] = 1 / dx
/tmp/ipykernel_622/1167035978.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  A[0, 2] = -1 / (4 * dx)
/tmp/ipykernel_622/1167035978.py:33: RuntimeWarning: divide by zero encountered in scalar divide
  A[-1, -1] = 3 / (4 * dx)
/tmp/ipykernel_622/1167035978.py:34: RuntimeWarning: divide by zero encountered in scalar divide
  A[-1, -2] = -1 / dx
/tmp/ipykernel_622/1167035978.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  A[-1, -3] = 1 / (4 * dx)
/tmp/ipykernel_622/1167035978.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  A[N_S-1, N_S-1-2] =  3 / (4 * dx)
/tmp/ipykernel_622/1167035978.py:38: RuntimeWarning: divide by zero encountered in scalar divide
  A[N_S-1, N_S-1-1] = -1 / dx
/tmp/ipyke